# 1 Setup

# 1.1 Read Packages

In [65]:
import pandas as pd
import numpy as np

from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k
from scipy import sparse, linalg

# 1.2 Read Data

In [2]:
df = pd.read_csv('UMW_FR_reduced.csv', index_col=0)
df_tags = pd.read_csv('tags_clean.csv', index_col=0)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (19,20,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


# 2 Modeling

# 2.1 Build Matrix A

In [69]:
#Drop na and duplicate rows
A = df[['user_id', 'mama_movie_id']].dropna()
A = A.drop_duplicates()
A['mama_movie_id'] = A['mama_movie_id'].astype(int)

In [70]:
#Limit the number of rows
A = A.iloc[:100000]

A['count'] = 1
A = A.pivot(index='user_id', columns='mama_movie_id', values='count')
A = A.fillna(0)

# 2.2 Build Matrix M

In [71]:
M = df_tags


# 2.3 Solve for Matrix C

In [75]:
movies = set.intersection(set(A.columns), set(df_tags.index))

A = A[movies]
A = A.values
#A = sparse.coo_matrix(A.values)

M = M.loc[movies]
M = M.values
#M = sparse.coo_matrix(M.values)

In [81]:
#???
C = A @ M @ linalg.inv(M.T @ M)

LinAlgError: singular matrix

# 2.4 Traditional Collabrative Filtering

In [50]:
model = LightFM(loss='warp', no_components=5)

In [51]:
model.fit(A, epochs=60, num_threads=8)

In [52]:
model.get_user_representations()[1]

array([[ 0.39998895, -0.65013313, -0.23270637, -0.45810607, -0.05914332],
       [ 0.6922737 , -0.12105871, -0.35075995, -0.48817614,  0.4662349 ],
       [ 0.43391943, -0.6149166 ,  0.5978549 ,  0.5740599 ,  0.5737708 ],
       ...,
       [ 0.03407878,  0.62354803,  0.45610762,  0.3683327 , -0.08827045],
       [-0.06707859, -0.0933042 ,  0.3046633 , -0.21722576,  0.573875  ],
       [ 0.20567076, -0.44873852,  0.12696955, -0.66888237,  0.9251136 ]],
      dtype=float32)

In [56]:
model.predict([0],[56])

array([-1.93970585])

In [54]:
A.todense()[0,50:60]

matrix([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]], dtype=float32)